# Covid 19 Data Analysis

## Covid-19 Daily Test Confirmation Rate in India

In [2]:
import requests
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
#import chart_studio.plotly as py
from plotly.offline import iplot

COVID19_INDIA_URL = 'https://api.covid19india.org/v3/data-all.json'

def get_covid_json():
    global URL

    r = requests.get(url = COVID19_INDIA_URL)
    jsonobj = r.json()
#    print(json.dumps(jsonobj, indent = 4))

    return jsonobj

def build_daily_metrics_india(jsonobj: dict):
    daily_metrics = {}
    for date in jsonobj:
        delta = jsonobj[date]['TT']['delta']
        if 'tested' in delta:
            daily_metrics[date] = {}
            daily_metrics[date]['confirmed'] = delta['confirmed']
            daily_metrics[date]['tested'] = delta['tested']
            daily_metrics[date]['test_confirm_rate'] = (delta['confirmed'] * 100) / delta['tested']
    return  daily_metrics

def get_df_metrics_india(jsonobj: dict):
    df = pd.DataFrame.from_dict(jsonobj, orient='index')
    df = df.rolling(window=7).mean().dropna()

    return df

jsonobj = get_covid_json()

daily_metrics = build_daily_metrics_india(jsonobj)
df = get_df_metrics_india(daily_metrics)

fig = px.line(df, x=df.index, y='test_confirm_rate')
fig.update_layout(
        title="7 Day Moving Average of Covid-19 Daily Test Confirmation Rate in India",
        yaxis_title="Test Confirmation Rate",
        xaxis_title="Date",
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
)

#fig.show()

# Creating trace1
#trace1 = go.Scatter(
#                    x = df.index,
#                    y = df.test_confirm_rate,
#                    mode = "lines",
#                    name = "India",
#                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
#                    text= "India")

#data = [trace1]
#layout = dict(title = '7 Day Moving Average of Covid-19 Daily Test Confirmation Rate in India',
#              xaxis= dict(title= 'Date',ticklen= 5,zeroline= False),
#              yaxis= dict(title= 'Test Confirmation Rate',ticklen= 5,zeroline= False)
#             )
#fig = dict(data = data, layout = layout)

iplot(fig)